In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import mne
from moabb.datasets import BNCI2014_001
import numpy as np
from ComfyNet import ComfyNet
from sklearn.model_selection import train_test_split
import torch
import random
from torch.utils.data import DataLoader
from torch import nn
from tqdm import tqdm
from BCIloader import get_bci_data, filter_bci_data

<frozen importlib._bootstrap>:241: FutureWarning: mne.io.pick.pick_types is deprecated will be removed in 1.6, use documented public API instead. If no appropriate public API exists, please open an issue on GitHub.
<frozen importlib._bootstrap>:241: FutureWarning: mne.io.pick.pick_channels_regexp is deprecated will be removed in 1.6, use documented public API instead. If no appropriate public API exists, please open an issue on GitHub.
<frozen importlib._bootstrap>:241: FutureWarning: mne.io.pick.channel_type is deprecated will be removed in 1.6, use documented public API instead. If no appropriate public API exists, please open an issue on GitHub.
d:\Programs\minconda3\envs\serpentine\Lib\site-packages\moabb\pipelines\__init__.py:26: ModuleNotFoundError: Tensorflow is not installed. You won't be able to use these MOABB pipelines if you attempt to do so.
  warn(
d:\Programs\minconda3\envs\serpentine\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupy

In [3]:
mne.set_config("MNE_DATA","bciData")

bci_subjects_excluded = []
bci_subjects = [i for i in range(1, 10) if i not in bci_subjects_excluded]

dataset = BNCI2014_001().get_data(subjects=bci_subjects)

In [4]:
bci_ch_names = dataset[1]['0train']['1'].ch_names
bci_n_channels = len(bci_ch_names)
bci_n_samples = dataset[1]['0train']['1'].n_times
bci_sfreq = 250

n_classes_bci = 2


In [5]:
X_train, y_train = get_bci_data(bci_subjects_excluded=[], test=False)
X_train = filter_bci_data(X_train, bci_sfreq=250)
X_test, y_test = get_bci_data(bci_subjects_excluded=[], test=True)
X_test = filter_bci_data(X_test, bci_sfreq=250)

Setting up band-pass filter from 4 - 40 Hz

IIR filter parameters
---------------------
Chebyshev I bandpass non-linear phase (one-pass forward) causal filter:
- Filter order 6 (forward)
- Cutoffs at 4.00, 40.00 Hz: -1.00, -1.00 dB



[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  24 tasks      | elapsed:    0.9s
[Parallel(n_jobs=2)]: Done 55520 tasks      | elapsed:    4.8s
[Parallel(n_jobs=2)]: Done 57024 out of 57024 | elapsed:    4.9s finished


Setting up band-pass filter from 4 - 40 Hz

IIR filter parameters
---------------------
Chebyshev I bandpass non-linear phase (one-pass forward) causal filter:
- Filter order 6 (forward)
- Cutoffs at 4.00, 40.00 Hz: -1.00, -1.00 dB



[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done 510 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 57024 out of 57024 | elapsed:    3.9s finished


In [6]:
class DatasetWrapped(torch.utils.data.Dataset):
    def __init__(self, X, Y):
        # Convert to torch tensors
        self.X = torch.from_numpy(X).float()
        self.Y = torch.from_numpy(Y).long()

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return [self.X[idx], self.Y[idx]]

train_dataset_bci = DatasetWrapped(X_train, y_train)
val_dataset_bci = DatasetWrapped(X_test, y_test)

train_dataloader = DataLoader(train_dataset_bci,batch_size = 32, shuffle = True)
test_dataloader = DataLoader(val_dataset_bci,batch_size = 32, shuffle= False)

In [7]:

n_channels = X_train.shape[1]
input_window_samples = X_train.shape[2]

model = ComfyNet(
    n_outputs=n_classes_bci,
    n_chans=len(bci_ch_names[:22]),
    n_filters_time=40,
    filter_time_length=32,
    pool_time_length=60,
    pool_time_stride=30,
    drop_prob=0.5,
    att_depth=2,
    att_heads=4,
    att_drop_prob=0.5,
    return_features=False,
    n_times = input_window_samples,
    final_fc_length = 760
)

# Display torchinfo table describing the model
print(model)


d:\Documents\Projects\Serpentine\comfynet-bci\utils\base.py:180: UserWarning: LogSoftmax final layer will be removed! Please adjust your loss function accordingly (e.g. CrossEntropyLoss)!
  warnings.warn("LogSoftmax final layer will be removed! " +


Layer (type (var_name):depth-idx)                            Input Shape               Output Shape              Param #                   Kernel Shape
ComfyNet (ComfyNet)                                          [1, 22, 640]              [1, 2]                    --                        --
├─_PatchEmbedding (patch_embedding): 1-1                     [1, 1, 22, 640]           [1, 19, 40]               --                        --
│    └─Sequential (shallownet): 2-1                          [1, 1, 22, 640]           [1, 40, 1, 19]            --                        --
│    │    └─Conv2d (0): 3-1                                  [1, 1, 22, 640]           [1, 40, 22, 609]          1,320                     [1, 32]
│    │    └─Conv2d (1): 3-2                                  [1, 40, 22, 609]          [1, 40, 1, 609]           35,240                    [22, 1]
│    │    └─BatchNorm2d (2): 3-3                             [1, 40, 1, 609]           [1, 40, 1, 609]           80             

d:\Programs\minconda3\envs\serpentine\Lib\site-packages\torch\nn\modules\conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ..\aten\src\ATen\native\cudnn\Conv_v8.cpp:919.)
  return F.conv2d(input, weight, bias, self.stride,


In [9]:
def train (model, optimizer, loss, train_dataloader, test_dataloder, epochs = 2, device = torch.device("cuda" if torch.cuda.is_available() else "cpu")):


    model.to(device)

    for epoch in range(epochs):
            
        total_train_loss = 0
        train_acc = 0
        total_val_loss = 0
        val_acc = 0
        avg_train_loss = 0
        avg_val_loss = 0
        avg_train_acc=0
        avg_val_acc = 0

        trained_samples = 0

        pbar = tqdm(train_dataloader)

        model.train()
        for i, (X, y) in enumerate(pbar):
            X, y = X.to(device), y.to(device)
            optimizer.zero_grad()
            y_pred = model(X)
            l = loss(y_pred, y)
            l.backward()
            optimizer.step()
            total_train_loss += l.item()
            train_acc += (y_pred.argmax(1) == y).sum().item()
            trained_samples+= X.shape[0]

            avg_train_loss = total_train_loss/trained_samples
            avg_train_acc = train_acc/trained_samples
            pbar.set_description(f"Epoch {epoch+1}, Loss: {avg_train_loss :.3f}, Accuracy: {avg_train_acc :.3f}")
        
        validated_samples = 0
        with torch.no_grad():
            for X, y in tqdm(test_dataloader):
                X, y = X.to(device), y.to(device)
                y_pred = model(X)
                l = loss(y_pred, y)
                total_val_loss += l.item()
                validated_samples += X.shape[0]
                val_acc += (y_pred.argmax(1) == y).sum().item()
                
                avg_val_loss = total_val_loss/validated_samples
                avg_val_acc = val_acc/validated_samples
            print(f"Validation Loss: {avg_val_loss :.3f}, Validation Accuracy: {avg_val_acc :.3f}")
        
        # wandb.log({'train_acc':avg_train_acc,"train_loss":avg_train_loss, "val_loss" : avg_val_loss, "val_acc": avg_val_acc})
        


In [10]:
optimizer = torch.optim.AdamW(model.parameters(), lr=0.0001)
loss = nn.CrossEntropyLoss()
train(model, optimizer, loss, train_dataloader, test_dataloader, epochs=10)

  0%|          | 0/81 [00:00<?, ?it/s]


RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [9]:
checkpoint = torch.load('models/model_7.pt')
checkpoint['val_acc']

0.8354978354978355